# 🛰️ Convolutional Autoencoder (CAE) on EuroSAT RGB Data

This notebook shows how to build and train a **Convolutional Autoencoder (CAE)** using PyTorch.
We will use the **EuroSAT RGB dataset** and visualize how images can be compressed and reconstructed.

In [ ]:
import os, zipfile, requests
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

## 1. Download and Load EuroSAT RGB Data

We will download a subset of the dataset (300 images per class) for this lesson.

In [ ]:
def download_eurosat():
    url = "https://madm.dfki.de/files/sentinel/EuroSAT.zip"
    zip_path = "EuroSAT.zip"
    target_dir = "EuroSAT/2750"

    if not os.path.exists(target_dir):
        print("⬇️ Downloading EuroSAT RGB dataset...")
        response = requests.get(url, stream=True, verify=False)
        total = int(response.headers.get('content-length', 0))
        with open(zip_path, 'wb') as f:
            downloaded = 0
            for data in response.iter_content(chunk_size=8192):
                f.write(data)
                downloaded += len(data)
                done = int(50 * downloaded / total)
                print(f"\r[{'=' * done}{' ' * (50 - done)}] {downloaded/1e6:.1f}/{total/1e6:.1f} MB", end='')
        print("\n📦 Extracting dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall("EuroSAT")
        os.remove(zip_path)
        print("✅ Dataset downloaded and extracted successfully!")
    else:
        print("✅ Dataset already available.")

def load_images(data_dir="EuroSAT/2750", max_per_class=300):
    images, labels, classes = [], [], []
    for class_dir in sorted(Path(data_dir).iterdir()):
        if class_dir.is_dir():
            cls = class_dir.name
            classes.append(cls)
            files = list(class_dir.glob("*.jpg"))[:max_per_class]
            for f in files:
                img = np.array(Image.open(f))
                images.append(img)
                labels.append(cls)
    return np.array(images), np.array(labels), classes

# Run download + load
download_eurosat()
images, true_labels, class_names = load_images(max_per_class=300)
print(f"✅ Loaded {len(images)} images from {len(class_names)} classes.")

## 2. Preprocessing for PyTorch

- Normalize images to [0,1]
- Convert to PyTorch tensors
- Create a DataLoader

In [ ]:
X_tensor = torch.tensor(images / 255.0, dtype=torch.float32).permute(0,3,1,2)  # (N,C,H,W)
dataset = TensorDataset(X_tensor, X_tensor)  # Autoencoder: input=target
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
print(f'Dataset shape: {X_tensor.shape}, Number of batches: {len(dataloader)}')

## 3. Build the Convolutional Autoencoder

- Encoder compresses images to latent vectors
- Decoder reconstructs images from latent vectors

In [ ]:
class ConvAutoencoder(nn.Module):
    def __init__(self, latent_dim=128):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1),  # 32x32x32
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),  # 64x16x16
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),  # 128x8x8
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128*8*8, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128*8*8),
            nn.Unflatten(1, (128,8,8)),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        z = self.encoder(x)
        out = self.decoder(z)
        return out

model = ConvAutoencoder(latent_dim=128).to(device)
print(model)

## 4. Train the Autoencoder

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 50
for epoch in range(n_epochs):
    epoch_loss = 0
    for batch, _ in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * batch.size(0)
    epoch_loss /= len(dataloader.dataset)
    print(f'Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss:.4f}')

## 5. Visualize Reconstructions

In [ ]:
model.eval()
with torch.no_grad():
    sample_imgs = X_tensor[:10].to(device)
    reconstructions = model(sample_imgs).cpu()

fig, axes = plt.subplots(2, 10, figsize=(20,4))
for i in range(10):
    axes[0,i].imshow(sample_imgs[i].permute(1,2,0).cpu())
    axes[0,i].axis('off')
    axes[0,i].set_title('Original')
    axes[1,i].imshow(reconstructions[i].permute(1,2,0))
    axes[1,i].axis('off')
    axes[1,i].set_title('Reconstructed')
plt.tight_layout()
plt.show()

## 6. Why CAEs are Important
- **Dimensionality reduction**: Latent vectors can be used for clustering, visualization, or classification.
- **Image compression**: Efficiently stores images with minimal quality loss.
- **GANs**: Encoders provide meaningful latent spaces for generative models.
- **Anomaly detection**: Reconstruction error highlights unusual patterns.